<a href="https://colab.research.google.com/github/Starky-16/ybi_intenship_project/blob/main/product_return_cost_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Product Return Cost Estimation

This project builds a machine learning model to estimate the cost associated with product returns for QuickKart, an e-commerce platform.

In [36]:

#import library
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_percentage_error




In [5]:
#read dataset
file_id = "1V_W59IZrTZklZPSFw6evY_T2J6uK2Dl0"
url = f"https://drive.google.com/uc?id={file_id}"

df = pd.read_csv(url)
df.head()

,product_id,product_type,return_reason,original_price,shipping_cost,refurbishment_cost,resale_value_loss_percent,resale_value_loss,total_return_cost
0,PID1000,Appliances,Damaged,13964,278.955627,477.501714,0.294416,4111.231126,4867.69
1,PID1001,Books,Wrong Item,8030,246.949265,376.604109,0.110172,884.677823,1508.23
2,PID1002,Books,Damaged,3176,273.708748,871.157883,0.380725,1209.183877,2354.05
3,PID1003,Electronics,Damaged,23227,59.904702,117.246806,0.360433,8371.787461,8548.94
4,PID1004,Clothing,Wrong Item,29712,250.235496,751.393581,0.286398,8509.460391,9511.09


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 1000 non-null   object 
 1   product_type               1000 non-null   object 
 2   return_reason              1000 non-null   object 
 3   original_price             1000 non-null   int64  
 4   shipping_cost              1000 non-null   float64
 5   refurbishment_cost         1000 non-null   float64
 6   resale_value_loss_percent  1000 non-null   float64
 7   resale_value_loss          1000 non-null   float64
 8   total_return_cost          1000 non-null   float64
dtypes: float64(5), int64(1), object(3)
memory usage: 70.4+ KB


In [7]:
df.describe(include='all')


,product_id,product_type,return_reason,original_price,shipping_cost,refurbishment_cost,resale_value_loss_percent,resale_value_loss,total_return_cost
count,1000,1000,1000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
unique,1000,5,5,NaN,NaN,NaN,NaN,NaN,NaN
top,PID1999,Books,Wrong Item,NaN,NaN,NaN,NaN,NaN,NaN
freq,1,215,220,NaN,NaN,NaN,NaN,NaN,NaN
mean,NaN,NaN,NaN,15391.144000,180.056955,521.370233,0.225857,3460.587719,4162.014990
std,NaN,NaN,NaN,8564.937302,72.223748,283.646716,0.098280,2567.546758,2580.570969
min,NaN,NaN,NaN,363.000000,50.045498,2.017644,0.050551,35.235308,377.540000
25%,NaN,NaN,NaN,8094.000000,116.025742,289.923801,0.140109,1314.706267,2107.732500
50%,NaN,NaN,NaN,15611.500000,182.641069,525.730416,0.227835,2851.170898,3523.495000
75%,NaN,NaN,NaN,23080.250000,244.530477,760.361468,0.308089,5044.305799,5808.452500


In [8]:
#encoding
df['product_type'].value_counts()


,count
product_type,
Books,215
Clothing,204
Appliances,198
Furniture,192
Electronics,191


In [33]:
df['product_type']=df['product_type'].replace({'Books':4,'Clothing':0,'Appliances':2,'Furniture':1,'Electronics':2})

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 1000 non-null   object 
 1   product_type               1000 non-null   int64  
 2   return_reason              1000 non-null   object 
 3   original_price             1000 non-null   int64  
 4   shipping_cost              1000 non-null   float64
 5   refurbishment_cost         1000 non-null   float64
 6   resale_value_loss_percent  1000 non-null   float64
 7   resale_value_loss          1000 non-null   float64
 8   total_return_cost          1000 non-null   float64
dtypes: float64(5), int64(2), object(2)
memory usage: 70.4+ KB


In [12]:
df['return_reason'].value_counts()


,count
return_reason,
Wrong Item,220
Size Mismatch,211
Damaged,197
Quality Issue,192
Changed Mind,180


In [32]:
df['return_reason']=df['return_reason'].replace({'Changed Mind':0,'Quality Issue':3,'Wrong Item':1,'Damaged':4,'Size Mismatch':2})

In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 1000 non-null   object 
 1   product_type               1000 non-null   int64  
 2   return_reason              1000 non-null   int64  
 3   original_price             1000 non-null   int64  
 4   shipping_cost              1000 non-null   float64
 5   refurbishment_cost         1000 non-null   float64
 6   resale_value_loss_percent  1000 non-null   float64
 7   resale_value_loss          1000 non-null   float64
 8   total_return_cost          1000 non-null   float64
dtypes: float64(5), int64(3), object(1)
memory usage: 70.4+ KB


In [16]:
df['product_type'] = pd.to_numeric(df['product_type'], errors='coerce').astype('Int64')


In [18]:
df.info()
df['product_id'].value_counts()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 1000 non-null   object 
 1   product_type               1000 non-null   Int64  
 2   return_reason              1000 non-null   int64  
 3   original_price             1000 non-null   int64  
 4   shipping_cost              1000 non-null   float64
 5   refurbishment_cost         1000 non-null   float64
 6   resale_value_loss_percent  1000 non-null   float64
 7   resale_value_loss          1000 non-null   float64
 8   total_return_cost          1000 non-null   float64
dtypes: Int64(1), float64(5), int64(2), object(1)
memory usage: 71.4+ KB


,count
product_id,
PID1999,1
PID1000,1
PID1001,1
PID1002,1
PID1003,1
...,...
PID1012,1
PID1011,1
PID1010,1


In [21]:
df['product_id'] = df['product_id'].astype(str).str.extract(r'(\d+)').astype(int)

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   product_id                 1000 non-null   int64  
 1   product_type               1000 non-null   Int64  
 2   return_reason              1000 non-null   int64  
 3   original_price             1000 non-null   int64  
 4   shipping_cost              1000 non-null   float64
 5   refurbishment_cost         1000 non-null   float64
 6   resale_value_loss_percent  1000 non-null   float64
 7   resale_value_loss          1000 non-null   float64
 8   total_return_cost          1000 non-null   float64
dtypes: Int64(1), float64(5), int64(3)
memory usage: 71.4 KB


In [22]:
#define x and y
df.columns
y = df['total_return_cost']
x=df[['product_id', 'product_type', 'return_reason', 'original_price',
       'shipping_cost', 'refurbishment_cost', 'resale_value_loss_percent',
       'resale_value_loss',]]

In [23]:
#traintest split
x_train,x_test,y_train,y_test=train_test_split(x,y,train_size=0.7)


In [24]:
#select model
model=LinearRegression()


In [25]:
# Train the model
model.fit(x_train, y_train)

# After training, get intercept and coefficients
model.intercept_,model.coef_

(np.float64(0.0010687126241464284),
 array([ 9.23691227e-08,  2.84400273e-05,  5.14181469e-05, -5.44592775e-08,
         9.99999636e-01,  9.99999944e-01, -4.25790019e-03,  1.00000019e+00]))

In [26]:
#prediction
y_pred=model.predict(x_test)

In [34]:
#evaluate
mean_absolute_percentage_error(y_test,y_pred)

8.821186495249154e-07

In [37]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
# Safeguard against divide-by-zero
y_test_safe = y_test.replace(0, np.finfo(float).eps)

# Calculate metrics
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
mape = mean_absolute_percentage_error(y_test_safe, y_pred) * 100

In [38]:
print("MAE:", mae)
print("RMSE:", rmse)
print("R² Score:", r2)
print("MAPE (%):", mape)

MAE: 0.002537308515451476
RMSE: 0.0029296499042539242
R² Score: 0.9999999999987284
MAPE (%): 8.821186495249153e-05
